In [432]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from pathlib import Path
import itertools
import sklearn
import sklearn.metrics
import io

In [433]:
BATCH_SIZE = 16
IMG_HEIGHT, IMG_WIDTH = (224, 224)
PREPROCESS_SEED = 123
CHECKPOINT_DIR = Path("checkpoints")
CHECKPOINT_PATH = CHECKPOINT_DIR / "cp-{epoch:04d}.ckpt"

In [434]:
base_data_dir = Path("..", "..", "input", "arch-recognizer-dataset").absolute()
val_data_dir = base_data_dir / "val"
test_data_dir = base_data_dir / "test"
train_data_dir = base_data_dir / "train"

In [435]:
# Import data

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    labels="inferred",
    label_mode="int",
    seed=PREPROCESS_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    crop_to_aspect_ratio=True,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="int",
    seed=PREPROCESS_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    crop_to_aspect_ratio=True,
)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="int",
    seed=PREPROCESS_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    crop_to_aspect_ratio=True,
)

class_names = train_ds.class_names

train_ds.map(lambda i, _: tf.keras.applications.resnet.preprocess_input(i))
val_ds.map(lambda i, _: tf.keras.applications.resnet.preprocess_input(i))
test_ds.map(lambda i, _: tf.keras.applications.resnet.preprocess_input(i))

train_ds.map(
    lambda i, _: tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(i)
)
val_ds.map(
    lambda i, _: tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(i)
)
test_ds.map(
    lambda i, _: tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(i)
)

train_ds = (
    train_ds.shuffle(5000, reshuffle_each_iteration=True)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
val_ds = (
    val_ds.shuffle(5000, reshuffle_each_iteration=True)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
test_ds = (
    test_ds
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)


Found 2021 files belonging to 25 classes.
Found 1012 files belonging to 25 classes.
Found 7080 files belonging to 25 classes.


In [436]:
# fashion_mnist = tf.keras.datasets.fashion_mnist

# (x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

In [437]:
# Configure Hyperparameters

HP_POOLING = hp.HParam(
    "pooling",
    hp.Discrete(
        [
            # None,
            "max",
            "avg",
        ]
    ),
)
HP_OPTIMIZER = hp.HParam(
    "optimizer",
    hp.Discrete(
        [
            # tf.keras.optimizers.Adam(learning_rate=0.0001),
            # tf.keras.optimizers.SGD(),
            "adam",
            "sgd",
        ]
    ),
)

METRIC_ACCURACY = "test_accuracy"

with tf.summary.create_file_writer(f"logs/hparam_tuning").as_default():
    hp.hparams_config(
        hparams=[
            # HP_WEIGHTS,
            HP_POOLING,
            HP_OPTIMIZER,
        ],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")],
    )


In [438]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

In [439]:
def restore_weights(model):
    latest_cp = tf.train.latest_checkpoint(CHECKPOINT_DIR)
    if latest_cp:
        model.load_weights(latest_cp)
        _, restored_test_acc = model.evaluate(test_ds, verbose=2)
        print(f"Restored model test accuracy: {restored_test_acc}")
    return model


def train_test_model(hparams, session_num):
    model = tf.keras.models.Sequential(
        [
            # Preprocessing
            tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),
            # Augmentation
            tf.keras.layers.experimental.preprocessing.RandomFlip(
                "horizontal", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
            ),
            tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
            tf.keras.applications.ResNet50(
                include_top=True,
                # weights=hparams[HP_WEIGHTS][1],
                weights=None,
                input_tensor=None,
                input_shape=None,
                # pooling=hparams[HP_POOLING][1],
                # classes=len(class_names),
            ),
        ]
    )

    restore_weights(model)

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    log_dir = Path("logs", f"run-{session_num}")

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=str(log_dir))

    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(len(class_names), len(class_names)))
        plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45, ha="right")
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype("float") / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.0
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format="png")
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        plt.show()
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image

    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(str(log_dir / "cm"))

    def log_confusion_matrix(epoch, logs):
        pred_labels, true_labels = [], []
        for batch_images, batch_labels in test_ds:  # use dataset.unbatch() with repeat
            true_labels.extend(batch_labels)
            preds = model.predict(batch_images)
            pred_labels.extend(np.argmax(preds, axis=-1))

        # # Convert into tensors
        # pred_labels_tensors = tf.concat([item for item in pred_labels], axis=0)
        # true_labels_tensors = tf.concat([item for item in true_labels], axis=0)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(true_labels, pred_labels)
        cm = np.nan_to_num(cm)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=class_names)
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    tb_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1, profile_batch=0
    )

    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    es_callback = (
        tf.keras.callbacks.EarlyStopping(
            min_delta=0.0001, patience=10, restore_best_weights=True
        ),
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,  
        callbacks=[tb_callback, cm_callback, es_callback],
    )
    _, accuracy = model.evaluate(test_ds)
    return accuracy


In [440]:
def run(run_dir, hparams, session_num):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [441]:
# Clear out any prior log data.
!rm -rf logs

# Load the TensorBoard notebook extension
%reload_ext tensorboard
%tensorboard --logdir logs --bind_all

# Perform training runs
session_num = 0
for pooling in HP_POOLING.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            # HP_WEIGHTS: weights,
            HP_POOLING: pooling,
            HP_OPTIMIZER: optimizer,
        }
        run_name = f"run-{session_num}"
        print(f"--- Starting trial: {run_name}")
        print({h.name: hparams[h] for h in hparams})
        run(f"logs/{run_name}", hparams, session_num)
        session_num += 1

--- Starting trial: run-0
{'pooling': 'avg', 'optimizer': 'adam'}


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/100
443/443 [==============================] - 134s 274ms/step - loss: 3.2539 - accuracy: 0.1066 - val_loss: 3.6879 - val_accuracy: 0.0782
Epoch 2/100
443/443 [==============================] - 112s 254ms/step - loss: 2.8851 - accuracy: 0.1486 - val_loss: 3.6581 - val_accuracy: 0.0891
